In [1]:
from training.model import SIREN
import torch
import numpy as np
from render.meshing import create_mesh
import open3d as o3d
import json
import meshplot as mp
import matplotlib.cm as cm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
FOLDER = 'segmented'
VERSION = ''
MESH = 'faust_54098'
EXPERIMENT = 'faust_54098'
N = 64
MODEL = 'best'

In [3]:
meshes_pred = []
meshes_gt = []
with open(f'../data/{FOLDER}/{MESH}{VERSION}.json') as jsonFile:
    skel = json.load(jsonFile)
    model = SIREN(
            n_in_features= len(skel['joints'][0]['mean']) + 3,
            n_out_features=1,
            hidden_layer_config=[256,256,256,256],
            w0=90,
            ww=None
    )
    model.load_state_dict( torch.load(f'../results/{MESH}/test_{EXPERIMENT}/models/model_{MODEL}.pth', map_location=torch.device(0)))
    
    for joint in skel['joints']:
        vertices, faces, normals, values = create_mesh(
            model,
            distance = torch.from_numpy(np.array(joint['mean']).astype(np.float32)),
            N=N,
            silent=True
        )
        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(vertices), o3d.utility.Vector3iVector(faces) )
        
        T = (np.block( [[np.eye(3,3), np.asarray(joint['position']).reshape((3,1))],[np.eye(1,4,k=3)] ] ) @ 
             np.block( [ [np.asarray(joint['base']), np.zeros((3,1))], [np.eye(1,4,k=3)]]) @ 
             np.diag( [1 / skel['scale']] * 3 + [1] ))
        
        mesh.transform( T )
        meshes_pred.append(mesh)

        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(np.array(joint['vertices']).astype(np.float32)), o3d.utility.Vector3iVector(np.array(joint['triangles']).astype(np.float32)) )
        mesh.transform( T )
        meshes_gt.append(mesh)


In [ ]:
p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_pred):
    if len(np.asarray(mesh.vertices)) > 0:
        if p is None:
            p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_pred)))[0:3])
        else:
            p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_pred)))[0:3] )
    else:
        print(i)

In [4]:
# exportar
for i, mesh in enumerate(meshes_pred):
    o3d.io.write_triangle_mesh(f'{i}.ply',mesh)

In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_gt):
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_gt)))[0:3])
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_gt)))[0:3] )

# CON INTERPOLACION

In [ ]:
from training.model import SIREN
import torch
import numpy as np
from render.meshing import create_mesh
import open3d as o3d
import json
import meshplot as mp
import matplotlib.cm as cm
import networkx as nx

In [ ]:
def readCodeGraph( path ):
    graph = nx.DiGraph()
    with open( path ) as jsonFile:
        skel = json.load(jsonFile)
        root = skel['root']
        scale = skel['scale']
        graph.add_nodes_from( [ (joint['id'], {'code': np.asarray(joint['mean']), 'base':np.asarray(joint['base']), 'position':np.asarray(joint['position'])}) for joint in skel['joints'] ] )

        for joint in skel['joints']:
            graph.add_edges_from( [ (joint['id'], child) for child in joint['children']] )

    return graph, root, scale

In [ ]:
class graphIterator:
    def __init__( self, graph: nx.DiGraph, root, samples=10 ):
        self.graph, self.root = graph, root
        self.branchGraph = self.removeDeg2Nodes()
        self.samples = samples

    def removeDeg2Nodes( self ):
        newGraph = nx.DiGraph()

        lastNode = []
        for node in nx.dfs_tree( self.graph, self.root ):
            if node == self.root:
                newGraph.add_node( node, code=self.graph.nodes[node]['code'] )
                lastNode = [node]
            elif self.graph.out_degree(node) > 1:
                newGraph.add_node( node, code=self.graph.nodes[node]['code'] )
                newGraph.add_edge( lastNode[0], node )
                lastNode = [node] * self.graph.out_degree(node) + lastNode[1:]
            elif self.graph.out_degree(node) == 0:
                newGraph.add_node( node, code=self.graph.nodes[node]['code'] )
                newGraph.add_edge( lastNode[0], node )
                lastNode = lastNode[1:]

        return newGraph

    def __iter__( self ):
        queue = []
        currentNode = self.root

        while True:
            if self.branchGraph.out_degree( currentNode ) != 0:
                for _, child in self.branchGraph.out_edges(currentNode):
                    branchIndex = np.argmax( np.abs( self.branchGraph.nodes[child]['code'] - self.branchGraph.nodes[currentNode]['code'] ) )
                    for i in np.linspace(0,1, self.samples, endpoint=False):
                        base, position = self.interpolate( currentNode, child, i, branchIndex )

                        sampledCode = np.array( self.branchGraph.nodes[currentNode]['code'] )
                        sampledCode[branchIndex] = i

                        yield sampledCode, base, position

                queue = [ nextNode for _, nextNode in self.branchGraph.out_edges(currentNode)] + queue
            else:
                yield self.graph.nodes[currentNode]['code'], self.graph.nodes[currentNode]['base'], self.graph.nodes[currentNode]['position']

            if len(queue) > 0:
                currentNode = queue.pop(0)
            else: 
                break

    def interpolate( self, nodeFrom, nodeTo, parameter, branchIndex ):
        path = nx.shortest_path( self.graph, nodeFrom, nodeTo )

        for i, node in enumerate(path):
            if np.isclose( self.graph.nodes[node]['code'][branchIndex], parameter):
                return self.graph.nodes[node]['base'], self.graph.nodes[node]['position']
            
            if self.graph.nodes[node]['code'][branchIndex] > parameter:
                previousNode = path[i-1]
                interpolatedPosition = (self.graph.nodes[node]['position'] - self.graph.nodes[previousNode]['position']) * (self.graph.nodes[node]['code'][branchIndex] - parameter) + self.graph.nodes[previousNode]['position']
                return (self.graph.nodes[node]['base'] + self.graph.nodes[previousNode]['base']) / 2, interpolatedPosition


        

In [ ]:
MESH = 'faust_54098'
EXPERIMENT = 'faust_54098_small'
N = 32
MODEL = 'best'

In [ ]:
branchGraph, root, scale = readCodeGraph(f'../results/{MESH}/{MESH}_t.json')
meshes_pred = []
model = SIREN(
        n_in_features= len(branchGraph.nodes[0]['code']) + 3,
        n_out_features=1,
        hidden_layer_config=[32,32,32,32],
        w0=30,
        ww=None
)
model.load_state_dict( torch.load(f'../results/{MESH}/test_{EXPERIMENT}/models/model_{MODEL}.pth', map_location=torch.device(0)))

In [ ]:
iter = graphIterator( branchGraph, root, 25 )
for code, base, position in iter:
    vertices, faces, normals, values = create_mesh(
        model,
        distance = torch.from_numpy(np.array(code)),
        N=N,
        silent=True
    )
    mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(vertices), o3d.utility.Vector3iVector(faces) )
    
    T = (np.block( [[np.eye(3,3), position.reshape((3,1))],[np.eye(1,4,k=3)] ] ) @ 
            np.block( [ [base, np.zeros((3,1))], [np.eye(1,4,k=3)]]) @ 
            np.diag( [1 / scale] * 3 + [1] ))
    
    mesh.transform( T )
    meshes_pred.append(mesh)

In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_pred):
    if len(np.asarray(mesh.vertices)) > 0:
        if p is None:
            p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_pred)))[0:3])
        else:
            p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_pred)))[0:3] )

# Solo json

In [ ]:
import numpy as np
import open3d as o3d
import json
import meshplot as mp
import matplotlib.cm as cm

In [ ]:
meshes_gt = []
with open(f'../data/segmented/faust_54098.json') as jsonFile:
    skel = json.load(jsonFile)
    
    for joint in skel['joints']:
        T = (np.block( [[np.eye(3,3), np.asarray(joint['position']).reshape((3,1))],[np.eye(1,4,k=3)] ] ) @ 
             np.block( [ [np.asarray(joint['base']), np.zeros((3,1))], [np.eye(1,4,k=3)]]) @ 
             np.diag( [1 / skel['scale']] * 3 + [1] ))
        
        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(np.array(joint['vertices']).astype(np.float32)), o3d.utility.Vector3iVector(np.array(joint['triangles']).astype(np.float32)) )
        mesh.transform( T )
        meshes_gt.append(mesh)


In [ ]:
p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_gt):
    if len(np.asarray(mesh.vertices)) > 0:
        if p is None:
            p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_gt)))[0:3])
        else:
            p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_gt)))[0:3] )
    else:
        print(i)